In [1]:
from collections import deque

class Graph:
    def __init__(self, size):
        self.adj_matrix = [[0] * size for _ in range(size)]
        self.size = size
        self.vertex_data = [''] * size

    def add_edge(self, u, v, c):
        if 0 <= u < self.size and 0 <= v < self.size:
            self.adj_matrix[u][v] = c
        else:
            raise ValueError("Vertex index out of bounds")

    def add_vertex_data(self, vertex, data):
        if 0 <= vertex < self.size:
            self.vertex_data[vertex] = data
        else:
            raise ValueError("Vertex index out of bounds")

    def bfs(self, s, t, parent):
        visited = [False] * self.size
        queue = deque([s])
        visited[s] = True

        while queue:
            u = queue.popleft()

            for ind, val in enumerate(self.adj_matrix[u]):
                if not visited[ind] and val > 0:
                    queue.append(ind)
                    visited[ind] = True
                    parent[ind] = u

        return visited[t]

    def edmonds_karp(self, source, sink):
        parent = [-1] * self.size
        max_flow = 0

        while self.bfs(source, sink, parent):
            path_flow = float("Inf")
            s = sink
            while s != source:
                path_flow = min(path_flow, self.adj_matrix[parent[s]][s])
                s = parent[s]

            max_flow += path_flow
            v = sink
            while v != source:
                u = parent[v]
                self.adj_matrix[u][v] -= path_flow
                self.adj_matrix[v][u] += path_flow
                v = parent[v]

            path = []
            v = sink
            while v != source:
                path.append(v)
                v = parent[v]
            path.append(source)
            path.reverse()
            path_names = [self.vertex_data[node] for node in path]
            print("Path:", " -> ".join(path_names), ", Flow:", path_flow)

        return max_flow

    def min_cut(self, source):
        visited = [False] * self.size
        queue = deque([source])
        visited[source] = True

        while queue:
            u = queue.popleft()
            for ind, val in enumerate(self.adj_matrix[u]):
                if not visited[ind] and val > 0:
                    queue.append(ind)
                    visited[ind] = True

        # Collecting the min cut edges
        min_cut_edges = []
        for u in range(self.size):
            for v in range(self.size):
                if visited[u] and not visited[v] and self.adj_matrix[u][v] == 0:
                    min_cut_edges.append((self.vertex_data[u], self.vertex_data[v]))

        return min_cut_edges

# Example usage:
g = Graph(6)
g.add_vertex_data(0, "Source")
g.add_vertex_data(1, "A")
g.add_vertex_data(2, "B")
g.add_vertex_data(3, "C")
g.add_vertex_data(4, "D")
g.add_vertex_data(5, "Sink")

g.add_edge(0, 1, 16)
g.add_edge(0, 2, 13)
g.add_edge(1, 2, 10)
g.add_edge(1, 3, 12)
g.add_edge(2, 1, 4)
g.add_edge(2, 4, 14)
g.add_edge(3, 2, 9)
g.add_edge(3, 5, 20)
g.add_edge(4, 3, 7)
g.add_edge(4, 5, 4)

max_flow = g.edmonds_karp(0, 5)
print("Max Flow:", max_flow)

min_cut_edges = g.min_cut(0)
print("Min Cut Edges:", min_cut_edges)


Path: Source -> A -> C -> Sink , Flow: 12
Path: Source -> B -> D -> Sink , Flow: 4
Path: Source -> B -> D -> C -> Sink , Flow: 7
Max Flow: 23
Min Cut Edges: [('Source', 'C'), ('Source', 'Sink'), ('A', 'C'), ('A', 'Sink'), ('B', 'C'), ('B', 'Sink'), ('D', 'C'), ('D', 'Sink')]
